<a href="https://colab.research.google.com/github/mistryvivek/YRKCS-PRBX/blob/main/Model_1_Basic_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [186]:
import torch
import torch.nn as nn
import numpy as np
import random
import pandas as pd
from torch.nn.utils.rnn import pad_sequence

In [187]:
class RNN(nn.Module):
    def __init__(self,max_laps,hidden_size):
        super(RNN, self).__init__()
        self.embedding =  nn.Embedding(max_laps, hidden_size)
        self.linear_h = nn.Linear(hidden_size,hidden_size)
        self.linear_y = nn.Linear(hidden_size,max_laps)
        self.tanh = nn.Tanh()
        # Make the existing model a binary classifier.
        self.sigmoid = nn.Sigmoid()

    def forward(self,x,hprev):
        h = self.tanh(self.embedding(x) + self.linear_h(hprev))
        y = self.sigmoid(self.linear_y(h))
        return h,y

# Loss function (Based upon wk11 practical)

* Vocab size is not relevent here - going to give it binary classification values anyway.
* Everything else can be applied for time series: use Cross Entropy Loss, Length of the race = seq_len.
* For each lap, pass through the RNN with previous hidden state and build up hte matrix of outputs.
* Detact the previous hidden state as a new one will be generated.

In [188]:
def calculate_loss(model,inputs,targets,hprev,max_tyre_life):
  loss_func = nn.BCEWithLogitsLoss()
  seq_length = len(inputs)
  #outputs = torch.zeros(0, max_laps)
  #print(outputs.shape)
  for t in range(seq_length):
    # For each character in the input sequence, pass through RNN with previous hidden state
    hprev,y = model(torch.tensor([inputs[t]]),hprev)
    # Gradually build up matrix of output logits of size seq_length * max_tyre_life
    #outputs = torch.cat((outputs ,y))

  # Compute cross entropy loss for seq_length actual targets against estimated distributions
  loss = loss_func(y,targets.unsqueeze(0))

  # For truncated backprop, the next subsequence will use the final hidden state
  # but will not backprop through it so we need to detach
  hprev = hprev.detach()

  return loss, hprev

# My dataset - how to format?

* Training and testing
* inputs_training/ inputs_testing/  outputs_testing/ output_training
* For now, it will all just be a 2d array.

ERROR:

```2551     return torch.embedding(weight, input, padding_idx, scale_grad_by_freq, sparse)
   2552
   2553

IndexError: index out of range in self```

In [189]:
BASE_PATH = r"/content/drive/MyDrive/prbx_data/v1/"
max_race_size = 0
max_tyre_life = 0

In [190]:
# TRAINING
training_inputs = []
training_outputs = []

RaceCalender22 = pd.read_csv(BASE_PATH + r"2022/eventCalender2022.csv")
for _, row in RaceCalender22.iterrows():
    if row['EventFormat'] != 'testing':
        # Load race data
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2022/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            # Filter data for the specific driver and sort by LapNumber
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            """if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])"""

            # Extract tyre life and stint columns
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values

            # Create the stint change array
            stint_change_array = []
            for i in range(len(stint_array)):
                if i == len(stint_array) - 1:  # Last lap
                    stint_change_array.append(False)  # Treat as no change
                else:
                    stint_change_array.append(stint_array[i] != stint_array[i + 1])

            stint_change_array = np.array(stint_change_array)

            # Ensure tyre_life_array and stint_change_array are properly formatted
            tyre_life_array = torch.tensor(tyre_life_array, dtype=torch.long)
            stint_change_array = torch.tensor(stint_change_array, dtype=torch.float32)
            # Add to training data
            training_inputs.append(tyre_life_array)
            training_outputs.append(stint_change_array)

In [191]:
# TESTING
testing_inputs = []
testing_outputs = []

RaceCalender23 = pd.read_csv(BASE_PATH + r"2023/eventCalender2023.csv")
for _, row in RaceCalender23.iterrows():
    if row['EventFormat'] != 'testing':
        # Load race data
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2023/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            # Filter data for the specific driver and sort by LapNumber
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            # Extract tyre life and stint columns
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values

            # Create the stint change array
            stint_change_array = []
            for i in range(len(stint_array)):
                if i == len(stint_array) - 1:  # Last lap
                    stint_change_array.append(False)  # Treat as no change
                else:
                    stint_change_array.append(stint_array[i] != stint_array[i + 1])

            stint_change_array = np.array(stint_change_array)

            # Ensure tyre_life_array and stint_change_array are properly formatted
            tyre_life_array = torch.tensor(tyre_life_array, dtype=torch.long)
            stint_change_array = torch.tensor(stint_change_array, dtype=torch.float32)

            # Add to testing data
            testing_inputs.append(tyre_life_array)
            testing_outputs.append(stint_change_array)

# Training the model

In [192]:
hidden_size = 1
lr = 0.1
iterations = 2000
max_race_size = int(max_race_size)

In [193]:
def pad_with_last_value(tensor, max_race_size):
    if len(tensor) > max_race_size:
        return

    last_value = tensor[-1]
    pad_size = max_race_size - len(tensor)
    padding = torch.full((pad_size,), last_value, dtype=tensor.dtype)
    return torch.cat((tensor, padding))

In [194]:
model = RNN(max_race_size, hidden_size)

optim = torch.optim.Adam(model.parameters(), lr = lr)

n, p = 0, 0
while n<=iterations:
  hprev = torch.zeros(hidden_size,) # reset RNN memory

  inputs = training_inputs[p]
  targets = training_outputs[p]

  inputs = pad_with_last_value(inputs,max_race_size)
  targets = pad_with_last_value(targets,max_race_size)


  # Compute loss for current subsequence
  loss, hprev = calculate_loss(model,inputs,targets,hprev,max_race_size)
  optim.zero_grad()
  loss.backward()
  optim.step()

  if n % 500 == 0:
    print('[{:}] Loss: {:.2f}'.format(n,loss.item()))

  n += 1 # iteration counter
  p += 1 # dataset counter

  #Restart to the front of the list.
  if p == len(training_inputs) -1:
    p = 0

[0] Loss: 0.95
[500] Loss: 0.69
[1000] Loss: 0.69
[1500] Loss: 0.69
[2000] Loss: 0.69


#How can we calculate accuracy for this basic model?

* Say is probability is above 80% percent pit, otherwise don't pit.
* Want at least one right pit stop identification to class it as match.

In [195]:
total, accurate_match = 0, 0
input_idx = 0
tyre_life_pits = []

for input_idx in range(len(testing_inputs) - 1):
  hprev = torch.zeros(hidden_size, )

  testing_input = testing_inputs[input_idx]
  testing_output = testing_outputs[input_idx]

  testing_input = pad_with_last_value(testing_input,max_race_size)
  testing_output = pad_with_last_value(testing_output,max_race_size)

  for t in range(len(testing_input) - 1):
      # For each character in the input sequence, pass through RNN with previous hidden state
      hprev,y = model(torch.tensor([testing_input[t]]),hprev)

  for output_idx in range(y.size(1)):
    if y[:, output_idx].item() > 0.5 and testing_output[output_idx].item() == 1:
      accurate_match += 1
    elif testing_output[output_idx].item() == 1:
      tyre_life_pits.append(testing_input[output_idx].item())


  total += 1

In [196]:
total, accurate_match

(437, 0)

In [197]:
import statistics as stats

stats.mean(tyre_life_pits), stats.mode(tyre_life_pits)

(16.917582417582416, 18)

In [198]:
total, accurate_match = 0, 0
input_idx = 0
tyre_life_pits = []

for input_idx in range(len(training_inputs) - 1):
  hprev = torch.zeros(hidden_size, )

  training_input = training_inputs[input_idx]
  training_output = training_outputs[input_idx]

  training_input = pad_with_last_value(testing_input,max_race_size)
  training_output = pad_with_last_value(testing_output,max_race_size)

  for t in range(len(training_input) - 1):
      # For each character in the input sequence, pass through RNN with previous hidden state
      hprev,y = model(torch.tensor([training_input[t]]),hprev)

  for output_idx in range(y.size(1)):
    if y[:, output_idx].item() > 0.5 and training_output[output_idx].item() == 1:
      accurate_match += 1
    elif training_output[output_idx].item() == 1:
      tyre_life_pits.append(training_input[output_idx].item())

  total += 1

In [199]:
stats.mean(tyre_life_pits), stats.mode(tyre_life_pits)

(29, 29)

In [200]:
tyre_life_pits

[29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,


# Plot to see what this basic model has remembered - probably just averages

# Validating my data

In [ ]:
len(training_inputs) == len(training_outputs)

True

In [ ]:
training_inputs[0].shape

torch.Size([54])

In [ ]:
training_outputs[0].shape

torch.Size([54])